In [20]:
import pandas as pd
import re
import numpy as np

In [21]:
CITIZEN_BANK_FILE = r"D:\For Auto Recon\BANK STATEMENT REPORT FOR RECONCILE Phase1-20230807T053815Z-001\BANK STATEMENT REPORT FOR RECONCILE Phase1\citizens 15-09-2020 to 15-07-2021.xlsx"
CITIZEN_SOA_FILE =r"D:\For Auto Recon\SOA REPORT FOR RECONCILE Phase1-20230807T053702Z-001\SOA REPORT FOR RECONCILE Phase1\Citizen Bank Commission Report (09_15_2020 to 07_15_2021)Citizen Bank.csv"

In [22]:
bank_statement_df = pd.read_excel(CITIZEN_BANK_FILE,skiprows=2)
soa_statement_df = pd.read_csv(CITIZEN_SOA_FILE,encoding='latin-1')

In [23]:
bank_statement_df.head()

,S. No,Tran Id,Sol Id,Init Sol Id,Foracid,Tran Date,Acct Name,Unnamed: 7,Acct Short Name,Dr Amount,Cr Amount,Instrmnt Num,Tran Particular,Tran Particular Two,Tran Remarks,Ref Num,Entered By,Posted By
0,1,S12645525,999,30,99901294081304,2020-09-15,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,100.0,NaN,MB/9843558137/100000029213/1600498,0300100000023031,1600498,000000515973,CDCI,CDCI
1,2,S12645585,999,1,99901294081304,2020-09-15,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,10.0,NaN,MB/9841476431/100000029218/1600514,0010100000298011,1600514,000000515990,CDCI,CDCI
2,3,S12663518,999,38,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,1000.0,NaN,MB/9851034423/100000029499/1601638,0380100000619032,1601638,000000517083,CDCI,CDCI
3,4,S12679714,999,45,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,2000.0,NaN,MB/9802548290/100000031053/1606746,0450100000072035,1606746,000000524073,CDCI,CDCI
4,5,S12665310,999,38,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,100.0,NaN,MB/9841798122/100000029723/1602486,0380100000256032,1602486,000000517923,CDCI,CDCI


In [24]:
soa_statement_df.head()

,Unnamed: 0,Transaction Id,Opening Balance,Mode,Transaction Type,Amount,Closing Blance,Remarks,Date
0,1,'271230,14292057.27,DR,Gateway,6500.0,14285557.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021 8:27:00 PM
1,2,'271237,14285557.27,DR,Gateway,200.0,14285357.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021 8:31:01 PM
2,3,'271338,14285357.27,DR,Gateway,500.0,14284857.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021 9:47:01 PM
3,4,'271360,14284857.27,DR,Gateway,20030.0,14264827.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021 10:34:00 PM
4,5,'271413,14264827.27,DR,Gateway,5000.0,14259827.27,Balance Transferred from Citizens(NPS Settleme...,1/27/2021 5:33:01 AM


In [25]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Tran Date'])
bank_statement_df['Date'] = bank_statement_df['Date'].dt.strftime('%m/%d/%y')

In [26]:
bank_statement_df.head()

,S. No,Tran Id,Sol Id,Init Sol Id,Foracid,Tran Date,Acct Name,Unnamed: 7,Acct Short Name,Dr Amount,Cr Amount,Instrmnt Num,Tran Particular,Tran Particular Two,Tran Remarks,Ref Num,Entered By,Posted By,Date
0,1,S12645525,999,30,99901294081304,2020-09-15,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,100.0,NaN,MB/9843558137/100000029213/1600498,0300100000023031,1600498,000000515973,CDCI,CDCI,09/15/20
1,2,S12645585,999,1,99901294081304,2020-09-15,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,10.0,NaN,MB/9841476431/100000029218/1600514,0010100000298011,1600514,000000515990,CDCI,CDCI,09/15/20
2,3,S12663518,999,38,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,1000.0,NaN,MB/9851034423/100000029499/1601638,0380100000619032,1601638,000000517083,CDCI,CDCI,09/16/20
3,4,S12679714,999,45,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,2000.0,NaN,MB/9802548290/100000031053/1606746,0450100000072035,1606746,000000524073,CDCI,CDCI,09/16/20
4,5,S12665310,999,38,99901294081304,2020-09-16,NPS SETTLEMENT ACCOUNT,NaN,NPS SET,0.0,100.0,NaN,MB/9841798122/100000029723/1602486,0380100000256032,1602486,000000517923,CDCI,CDCI,09/16/20


In [27]:
for index, row in bank_statement_df.iterrows():
            extracted_number_1000000 = re.findall(r'1000000(\d{5})', str(row["Tran Particular"]))
            extracted_number_100000 = re.findall(r'100000(\d{6})', str(row["Tran Particular"]))

            if extracted_number_1000000:
                extracted_number = extracted_number_1000000[0]
                bank_statement_df.at[index, 'Transaction Id'] = extracted_number
            elif extracted_number_100000:
                extracted_number = extracted_number_100000[0]
                bank_statement_df.at[index, 'Transaction Id'] = extracted_number
                
            particular_str = row["Tran Particular"]
            cips_matches = re.findall(r'cIPS/\d+/(\d+)/\d+', particular_str)
            
            if not cips_matches:
                continue
            transaction_id = cips_matches[0]
            bank_statement_df.loc[index, 'Transaction Id'] = transaction_id
print(f"total_tid {bank_statement_df['Transaction Id'].count()}")
print(f"transaction_id ====>{bank_statement_df['Transaction Id']}")

total_tid 12344
transaction_id ====>0         29213
1         29218
2         29499
3         31053
4         29723
          ...  
12668    731621
12669    730540
12670    728198
12671    728413
12672    728360
Name: Transaction Id, Length: 12673, dtype: object


In [28]:
soa_statement_df['Transaction Id'] =soa_statement_df['Transaction Id'].str.strip("'").dropna()
soa_statement_df['Date'] = soa_statement_df['Date'].astype(str)
for index, date in enumerate(soa_statement_df['Date']):
    soa_statement_df.loc[index, 'Date'] = date.split(' ')[0]

In [29]:
soa_statement_df.head()

,Unnamed: 0,Transaction Id,Opening Balance,Mode,Transaction Type,Amount,Closing Blance,Remarks,Date
0,1,271230,14292057.27,DR,Gateway,6500.0,14285557.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021
1,2,271237,14285557.27,DR,Gateway,200.0,14285357.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021
2,3,271338,14285357.27,DR,Gateway,500.0,14284857.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021
3,4,271360,14284857.27,DR,Gateway,20030.0,14264827.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021
4,5,271413,14264827.27,DR,Gateway,5000.0,14259827.27,Balance Transferred from Citizens(NPS Settleme...,1/27/2021


In [30]:
for index1 , bank_statement in bank_statement_df.iterrows():
    if (bank_statement['Transaction Id']) in (soa_statement_df['Transaction Id'].values):
        bank_statement_df.loc[index1,'Matched_TID'] = bank_statement['Transaction Id']
        continue
    bank_statement_df.loc[index1,'Unmatched_TID'] = bank_statement['Transaction Id']
    
  
print(bank_statement_df['Unmatched_TID'].count())  
print(bank_statement_df['Matched_TID'].count())

1124
11220


In [31]:
for index1, bank_statement_soa in soa_statement_df.iterrows():
    transaction_id = bank_statement_soa['Transaction Id']

    if transaction_id in bank_statement_df['Transaction Id'].values:
        soa_statement_df.loc[index1,'Matched_tid'] = transaction_id
        continue
    soa_statement_df.loc[index1,'Unmatched_tid'] = transaction_id

print(soa_statement_df['Matched_tid'].count())
print(soa_statement_df['Unmatched_tid'].count())

11297
1402


In [32]:
def merge_soa_matched_column(soa_statement_df):
        soa_statement_df['Matched'] = soa_statement_df[['Matched_TID', 'Matched_AD']].any(axis=1)
        # duplicate_rows = self.soa_statement_df[self.soa_statement_df.duplicated(subset=['Date', 'Amount'], keep=False)]
        # self.soa_statement_df.loc[duplicate_rows.index, 'Matched'] = False
        soa_statement_df['Matched'] = soa_statement_df[['Matched_TID', 'Matched_AD']].any(axis=1)

        matched_ad_true_rows = soa_statement_df[soa_statement_df['Matched_AD'] == True]
        duplicate_rows = matched_ad_true_rows[matched_ad_true_rows.duplicated(subset=['Date', 'Amount'], keep=False)]
        soa_statement_df.loc[duplicate_rows.index, 'Matched'] = False

        match_count = (soa_statement_df['Matched'] == True).sum()
        unmatch_count = (soa_statement_df['Matched'] == False).sum()
        display(f'Matched final count soa_statement  ==== {match_count}')
        display(f'Unmatched final count soa_statement  ==== {unmatch_count}')

In [33]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Tran Date'])
bank_statement_df['Date'] 

0       2020-09-15
1       2020-09-15
2       2020-09-16
3       2020-09-16
4       2020-09-16
           ...    
12668   2021-07-15
12669   2021-07-15
12670   2021-07-15
12671   2021-07-15
12672   2021-07-15
Name: Date, Length: 12673, dtype: datetime64[ns]

In [34]:
bank_statement_df['Date'][1]

Timestamp('2020-09-15 00:00:00')

In [35]:
soa_statement_df.head()

,Unnamed: 0,Transaction Id,Opening Balance,Mode,Transaction Type,Amount,Closing Blance,Remarks,Date,Matched_tid,Unmatched_tid
0,1,271230,14292057.27,DR,Gateway,6500.0,14285557.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021,271230,NaN
1,2,271237,14285557.27,DR,Gateway,200.0,14285357.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021,271237,NaN
2,3,271338,14285357.27,DR,Gateway,500.0,14284857.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021,271338,NaN
3,4,271360,14284857.27,DR,Gateway,20030.0,14264827.27,Balance Transferred from Citizens(NPS Settleme...,1/26/2021,271360,NaN
4,5,271413,14264827.27,DR,Gateway,5000.0,14259827.27,Balance Transferred from Citizens(NPS Settleme...,1/27/2021,271413,NaN


In [36]:
soa_statement_df['Date'] = pd.to_datetime(soa_statement_df['Date'], format="%m/%d/%Y")
soa_statement_df['Date']

0       2021-01-26
1       2021-01-26
2       2021-01-26
3       2021-01-26
4       2021-01-27
           ...    
12695   2021-01-26
12696   2021-01-26
12697   2021-01-26
12698   2021-01-26
12699          NaT
Name: Date, Length: 12700, dtype: datetime64[ns]